# TensorFlow__MLP_MNIST-labeled100__pollenjp

- 論文
  - [Semi-Supervised Learning with Ladder Networks - Antti Rasmus, Harri Valpola, Mikko Honkala, Mathias Berglund, Tapani Raiko](https://arxiv.org/abs/1507.02672)

- code
  - [rinuboney/ladder](https://github.com/rinuboney/ladder)
  - [tarvaina/tensorflow-ladder](https://github.com/tarvaina/tensorflow-ladder)


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Config" data-toc-modified-id="Config-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Config</a></span><ul class="toc-item"><li><span><a href="#Import" data-toc-modified-id="Import-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Import</a></span></li><li><span><a href="#TensorFlow,-Keras" data-toc-modified-id="TensorFlow,-Keras-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>TensorFlow, Keras</a></span></li><li><span><a href="#Path" data-toc-modified-id="Path-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Path</a></span></li><li><span><a href="#Import-from-Local" data-toc-modified-id="Import-from-Local-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Import from Local</a></span></li></ul></li><li><span><a href="#Load-MNIST" data-toc-modified-id="Load-MNIST-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load MNIST</a></span></li><li><span><a href="#Simple-MLP" data-toc-modified-id="Simple-MLP-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Simple MLP</a></span><ul class="toc-item"><li><span><a href="#fit" data-toc-modified-id="fit-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>fit</a></span></li><li><span><a href="#Show-Computational-Graph" data-toc-modified-id="Show-Computational-Graph-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Show Computational Graph</a></span></li></ul></li><li><span><a href="#MLP-(Batch-Normalization)" data-toc-modified-id="MLP-(Batch-Normalization)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>MLP (Batch Normalization)</a></span><ul class="toc-item"><li><span><a href="#Session" data-toc-modified-id="Session-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Session</a></span></li></ul></li><li><span><a href="#show-Result" data-toc-modified-id="show-Result-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>show Result</a></span></li><li><span><a href="#End" data-toc-modified-id="End-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>End</a></span></li></ul></div>

## Config

### Import

In [1]:
import os,sys
print(sys.version)

import re
from pathlib import Path
import math

3.5.2 (default, Nov 23 2017, 16:37:01) 
[GCC 5.4.0 20160609]


In [2]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2
# If you want to reload manually, add a below line head.
%aimport
# ref: https://ipython.org/ipython-doc/3/config/extensions/autoreload.html

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import sklearn
from sklearn import datasets
import tqdm
import csv
import pandas as pd

seed = None
np.random.seed(seed=seed)

print("numpy ver: {}".format(np.__version__))
print("scikit-learn ver: {}".format(sklearn.__version__))
print("pandas ver: {}".format(pd.__version__))

Modules to reload:
all-except-skipped

Modules to skip:

numpy ver: 1.15.1
scikit-learn ver: 0.19.2
pandas ver: 0.23.4


### TensorFlow, Keras

In [3]:
#____________________________________________________________________________________________________
#  TensorFlow and Keras GPU configures
##________________________________________________________________________________
##  OPTIONAL : set a GPU viewed by TensorFlow
###____________________________________________________________
###  - https://stackoverflow.com/questions/37893755/tensorflow-set-cuda-visible-devices-within-jupyter
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"
##________________________________________________________________________________


##________________________________________________________________________________
##  TensorFlow
###____________________________________________________________
import tensorflow as tf
print("tensorflow ver: {}".format(tf.__version__))
###  eager mode
#tf.enable_eager_execution()
print("tf.executing_eagerly(): {}".format(tf.executing_eagerly()))

# You can double check that you have the correct devices visible to TF
#   - https://stackoverflow.com/questions/37893755/tensorflow-set-cuda-visible-devices-within-jupyter
from tensorflow.python.client import device_lib
print("""
________________________________________
Visible GPUs from TensorFlow
________________________________________""")
for _device in device_lib.list_local_devices():
    match = re.search(pattern=r'name: "/device:(?P<name>[A-Z]{3}):(?P<device_num>\d{1})*',
                      string=str(_device))
    if match is None:
        print("Not Match")
        continue
    if match.group("name") == "CPU":
        name, device_num = match.group("name", "device_num")
        print()
        print("({}:{})".format(name, device_num))
        continue
    name, device_num = match.group("name", "device_num")
    match = re.search(pattern=r'.*pci bus id: (?P<pci_bus_id>\d{4}:\d{2}:\d{2}.\d{1}).*',
                      string=str(_device))
    if match is None:
        print("No GPUs")
        continue
    print("({}:{}: pci_bus_id: {})".format(name, device_num, match.group("pci_bus_id")))
print("________________________________________")

###____________________________________________________________
###  sessioin
global _SESSION
config = tf.ConfigProto(allow_soft_placement=True,
                        log_device_placement=True)
config.gpu_options.allow_growth = True
_SESSION = tf.Session(config=config)
###____________________________________________________________
##________________________________________________________________________________
#____________________________________________________________________________________________________

import keras
import keras.backend as K
config = tf.ConfigProto(allow_soft_placement=True,
                        log_device_placement=True)
config.gpu_options.allow_growth = True
_SESSION = tf.Session(config=config)
K.set_session(_SESSION)

tensorflow ver: 1.10.1
tf.executing_eagerly(): False

________________________________________
Visible GPUs from TensorFlow
________________________________________

(CPU:0)
(GPU:0: pci_bus_id: 0000:02:00.0)
________________________________________


Using TensorFlow backend.


### Path

In [4]:
HOME = Path(os.getcwd()).parent
print(HOME)

/home/pollenjp/workdir/git/article_script/20180914__semi-supervised-deeplearning-ladder-networks__in_kabuku


In [5]:
path_list = []
data_Path = HOME / "data"
path_list.append(data_Path)
raw_Path = data_Path / "raw"
path_list.append(raw_Path)
plot_images_Path = data_Path / "plot_images"
path_list.append(plot_images_Path)
src_Path = HOME / "src"
path_list.append(src_Path)

for _Path in path_list:
    _path = str(_Path)
    if not os.path.exists(_path):
        os.makedirs(name=_path)
        print("make a directory: \n\t", _path)
    else:
        print(os.path.exists(_path), ": ", _path)

True :  /home/pollenjp/workdir/git/article_script/20180914__semi-supervised-deeplearning-ladder-networks__in_kabuku/data
True :  /home/pollenjp/workdir/git/article_script/20180914__semi-supervised-deeplearning-ladder-networks__in_kabuku/data/raw
True :  /home/pollenjp/workdir/git/article_script/20180914__semi-supervised-deeplearning-ladder-networks__in_kabuku/data/plot_images
True :  /home/pollenjp/workdir/git/article_script/20180914__semi-supervised-deeplearning-ladder-networks__in_kabuku/src


### Import from Local

In [6]:
sys.path.append(str(src_Path))

In [7]:
from data import mnist_input_data
from utils_tensorflow.tensorflow_graph_in_jupyer import show_computational_graph

## Load MNIST

In [8]:
num_labeled = 100
print( "===  Loading Data ===" )
mnist = mnist_input_data.read_data_sets(train_dir=str(raw_Path / "MNIST_data"), n_labeled=num_labeled,
                                        fake_data=False, one_hot=True)

===  Loading Data ===
Extracting /home/pollenjp/workdir/git/article_script/20180914__semi-supervised-deeplearning-ladder-networks__in_kabuku/data/raw/MNIST_data/train-images-idx3-ubyte.gz
Extracting /home/pollenjp/workdir/git/article_script/20180914__semi-supervised-deeplearning-ladder-networks__in_kabuku/data/raw/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /home/pollenjp/workdir/git/article_script/20180914__semi-supervised-deeplearning-ladder-networks__in_kabuku/data/raw/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /home/pollenjp/workdir/git/article_script/20180914__semi-supervised-deeplearning-ladder-networks__in_kabuku/data/raw/MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
X_train = mnist.semi_train.labeled_ds.images
y_train = mnist.semi_train.labeled_ds.labels
X_test = mnist.semi_test.images
y_test = mnist.semi_test.labels
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(100, 784)
(100, 10)
(10000, 784)
(10000, 10)


In [10]:
y_train[:10]
y_train.sum(axis=0)

array([10., 10., 10., 10., 10., 10., 10., 10., 10., 10.])

## Simple MLP

In [11]:
layer_sizes = [784, 1000, 500, 250, 250, 250, 10]
L = len(layer_sizes) - 1  # number of layers

num_examples = 60000
num_epochs = 300

starter_learning_rate = 0.02
decay_after = 15  # epoch after which to begin learning rate decay
batch_size = 100
num_iter = (num_examples//batch_size) * num_epochs # number of loop iterations

In [12]:
from keras.models import Sequential
from keras import layers
from keras import activations

In [13]:
K.clear_session()
layer_sizes = [784, 1000, 500, 250, 250, 250, 10]
model = Sequential()
model.add(layer=layers.Dense(units=layer_sizes[1], activation="relu", input_shape=(layer_sizes[0], )))
model.add(layer=layers.BatchNormalization())
model.add(layer=layers.Dense(units=layer_sizes[2], activation="relu"))
model.add(layer=layers.BatchNormalization())
model.add(layer=layers.Dense(units=layer_sizes[3], activation="relu"))
model.add(layer=layers.BatchNormalization())
model.add(layer=layers.Dense(units=layer_sizes[4], activation="relu"))
model.add(layer=layers.BatchNormalization())
model.add(layer=layers.Dense(units=layer_sizes[5], activation="relu"))
model.add(layer=layers.BatchNormalization())
model.add(layer=layers.Dense(units=layer_sizes[6], activation="softmax"))

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              785000    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1000)              4000      
_________________________________________________________________
dense_2 (Dense)              (None, 500)               500500    
_________________________________________________________________
batch_normalization_2 (Batch (None, 500)               2000      
_________________________________________________________________
dense_3 (Dense)              (None, 250)               125250    
_________________________________________________________________
batch_normalization_3 (Batch (None, 250)               1000      
_________________________________________________________________
dense_4 (Dense)              (None, 250)               62750     
__________

In [15]:
from keras import optimizers

In [16]:
learning_rate = 0.02
metrics = ["accuracy"]
opt = optimizers.Adam(lr=learning_rate)
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=metrics)

### fit

In [17]:
model.fit(x=X_train, y=y_train, batch_size=batch_size, epochs=num_epochs,
          validation_data=(X_test[:], y_test[:]) )

Train on 100 samples, validate on 10000 samples
Epoch 1/300
100/100 [==============================] - 1s 13ms/step - loss: 2.9616 - acc: 0.1100 - val_loss: 10.6892 - val_acc: 0.2676
Epoch 2/300
100/100 [==============================] - 0s 995us/step - loss: 1.0904 - acc: 0.6400 - val_loss: 8.3102 - val_acc: 0.3977
Epoch 3/300
100/100 [==============================] - 0s 1ms/step - loss: 1.0202 - acc: 0.7500 - val_loss: 7.9438 - val_acc: 0.4422
Epoch 4/300
100/100 [==============================] - 0s 999us/step - loss: 0.1898 - acc: 0.9500 - val_loss: 8.8905 - val_acc: 0.3908
Epoch 5/300
100/100 [==============================] - 0s 1ms/step - loss: 0.1243 - acc: 0.9400 - val_loss: 9.1905 - val_acc: 0.3655
Epoch 6/300
100/100 [==============================] - 0s 999us/step - loss: 0.0336 - acc: 0.9900 - val_loss: 8.8903 - val_acc: 0.3698
Epoch 7/300
100/100 [==============================] - 0s 1ms/step - loss: 0.0116 - acc: 1.0000 - val_loss: 8.3459 - val_acc: 0.3929
Epoch 8/300
1

Epoch 61/300
100/100 [==============================] - 0s 1ms/step - loss: 4.9149e-05 - acc: 1.0000 - val_loss: 2.2847 - val_acc: 0.7072
Epoch 62/300
100/100 [==============================] - 0s 999us/step - loss: 4.8092e-05 - acc: 1.0000 - val_loss: 2.2706 - val_acc: 0.7078
Epoch 63/300
100/100 [==============================] - 0s 1ms/step - loss: 4.7094e-05 - acc: 1.0000 - val_loss: 2.2571 - val_acc: 0.7087
Epoch 64/300
100/100 [==============================] - 0s 1ms/step - loss: 4.6142e-05 - acc: 1.0000 - val_loss: 2.2441 - val_acc: 0.7088
Epoch 65/300
100/100 [==============================] - 0s 1ms/step - loss: 4.5239e-05 - acc: 1.0000 - val_loss: 2.2317 - val_acc: 0.7086
Epoch 66/300
100/100 [==============================] - 0s 999us/step - loss: 4.4372e-05 - acc: 1.0000 - val_loss: 2.2198 - val_acc: 0.7090
Epoch 67/300
100/100 [==============================] - 0s 999us/step - loss: 4.3565e-05 - acc: 1.0000 - val_loss: 2.2082 - val_acc: 0.7085
Epoch 68/300
100/100 [======

Epoch 121/300
100/100 [==============================] - 0s 1ms/step - loss: 2.3092e-05 - acc: 1.0000 - val_loss: 1.9287 - val_acc: 0.7062
Epoch 122/300
100/100 [==============================] - 0s 1ms/step - loss: 2.2892e-05 - acc: 1.0000 - val_loss: 1.9266 - val_acc: 0.7060
Epoch 123/300
100/100 [==============================] - 0s 1ms/step - loss: 2.2699e-05 - acc: 1.0000 - val_loss: 1.9246 - val_acc: 0.7061
Epoch 124/300
100/100 [==============================] - 0s 1ms/step - loss: 2.2508e-05 - acc: 1.0000 - val_loss: 1.9226 - val_acc: 0.7062
Epoch 125/300
100/100 [==============================] - 0s 999us/step - loss: 2.2323e-05 - acc: 1.0000 - val_loss: 1.9206 - val_acc: 0.7057
Epoch 126/300
100/100 [==============================] - 0s 1000us/step - loss: 2.2140e-05 - acc: 1.0000 - val_loss: 1.9187 - val_acc: 0.7058
Epoch 127/300
100/100 [==============================] - 0s 1ms/step - loss: 2.1950e-05 - acc: 1.0000 - val_loss: 1.9169 - val_acc: 0.7056
Epoch 128/300
100/100 

Epoch 180/300
100/100 [==============================] - 0s 1ms/step - loss: 1.5075e-05 - acc: 1.0000 - val_loss: 1.8579 - val_acc: 0.7019
Epoch 181/300
100/100 [==============================] - 0s 1ms/step - loss: 1.4979e-05 - acc: 1.0000 - val_loss: 1.8573 - val_acc: 0.7018
Epoch 182/300
100/100 [==============================] - 0s 1ms/step - loss: 1.4892e-05 - acc: 1.0000 - val_loss: 1.8567 - val_acc: 0.7018
Epoch 183/300
100/100 [==============================] - 0s 1ms/step - loss: 1.4794e-05 - acc: 1.0000 - val_loss: 1.8561 - val_acc: 0.7018
Epoch 184/300
100/100 [==============================] - 0s 1ms/step - loss: 1.4710e-05 - acc: 1.0000 - val_loss: 1.8554 - val_acc: 0.7016
Epoch 185/300
100/100 [==============================] - 0s 1ms/step - loss: 1.4615e-05 - acc: 1.0000 - val_loss: 1.8549 - val_acc: 0.7016
Epoch 186/300
100/100 [==============================] - 0s 1ms/step - loss: 1.4532e-05 - acc: 1.0000 - val_loss: 1.8543 - val_acc: 0.7017
Epoch 187/300
100/100 [====

Epoch 239/300
100/100 [==============================] - 0s 1ms/step - loss: 1.0898e-05 - acc: 1.0000 - val_loss: 1.8338 - val_acc: 0.7009
Epoch 240/300
100/100 [==============================] - 0s 1ms/step - loss: 1.0843e-05 - acc: 1.0000 - val_loss: 1.8336 - val_acc: 0.7009
Epoch 241/300
100/100 [==============================] - 0s 1ms/step - loss: 1.0793e-05 - acc: 1.0000 - val_loss: 1.8333 - val_acc: 0.7009
Epoch 242/300
100/100 [==============================] - 0s 1ms/step - loss: 1.0740e-05 - acc: 1.0000 - val_loss: 1.8331 - val_acc: 0.7009
Epoch 243/300
100/100 [==============================] - 0s 1ms/step - loss: 1.0688e-05 - acc: 1.0000 - val_loss: 1.8329 - val_acc: 0.7010
Epoch 244/300
100/100 [==============================] - 0s 1ms/step - loss: 1.0637e-05 - acc: 1.0000 - val_loss: 1.8326 - val_acc: 0.7010
Epoch 245/300
100/100 [==============================] - 0s 1ms/step - loss: 1.0587e-05 - acc: 1.0000 - val_loss: 1.8324 - val_acc: 0.7008
Epoch 246/300
100/100 [====

Epoch 298/300
100/100 [==============================] - 0s 1ms/step - loss: 8.3501e-06 - acc: 1.0000 - val_loss: 1.8242 - val_acc: 0.7017
Epoch 299/300
100/100 [==============================] - 0s 1ms/step - loss: 8.3197e-06 - acc: 1.0000 - val_loss: 1.8241 - val_acc: 0.7017
Epoch 300/300
100/100 [==============================] - 0s 1ms/step - loss: 8.2887e-06 - acc: 1.0000 - val_loss: 1.8240 - val_acc: 0.7017


### Show Computational Graph

In [18]:
show_computational_graph(graph_def=tf.get_default_graph())

## MLP (Batch Normalization)

In [27]:
K.clear_session()

In [28]:
layer_sizes = [784, 1000, 500, 250, 250, 250, 10]
L = len(layer_sizes) - 1  # number of layers

num_examples = 60000
num_epochs = 150

starter_learning_rate = 0.02
decay_after = 15  # epoch after which to begin learning rate decay
batch_size = 100
num_iter = (num_examples//batch_size) * num_epochs # number of loop iterations

### Session

In [31]:
print( "===  Starting Session ===" )
sess = tf.Session()

===  Starting Session ===


In [41]:
i_iter = 0

ckpt = tf.train.get_checkpoint_state('checkpoints/')  # get latest checkpoint (if any)
if ckpt and ckpt.model_checkpoint_path:
    # if checkpoint exists, restore the parameters and set epoch_n and i_iter
    saver.restore(sess, ckpt.model_checkpoint_path)
    epoch_n = int(ckpt.model_checkpoint_path.split('-')[1])
    # EDITED
    # https://github.com/rinuboney/ladder/issues/12#issuecomment-332497271
    #i_iter = (epoch_n+1) * (num_examples/batch_size)
    i_iter = (epoch_n+1) * (num_examples//batch_size)
    print( "Restored Epoch ", epoch_n )
else:
    # no checkpoint exists. create checkpoints directory if it does not exist.
    if not os.path.exists('checkpoints'):
        os.makedirs('checkpoints')
    init = tf.global_variables_initializer()
    sess.run(init)

print( "=== Training ===" )
print( "Initial Accuracy: {}%", sess.run(accuracy,
                                      feed_dict={
                                          inputs: mnist.semi_test.images,
                                          outputs: mnist.semi_test.labels,
                                          training: False}) )

NameError: name 'sess' is not defined

In [ ]:
for i in tqdm.tqdm(range(i_iter, num_iter)):
    images, labels = mnist.semi_train.next_batch(batch_size)
    sess.run(train_step, feed_dict={inputs: images, outputs: labels, training: True})
    if (i > 1) and ((i+1) % (num_iter//num_epochs) == 0):
        epoch_n = i//(num_examples//batch_size)
        if (epoch_n+1) >= decay_after:
            # decay learning rate
            # learning_rate = starter_learning_rate * ((num_epochs - epoch_n) / (num_epochs - decay_after))
            ratio = 1.0 * (num_epochs - (epoch_n+1))  # epoch_n + 1 because learning rate is set for next epoch
            ratio = max(0, ratio / (num_epochs - decay_after))
            sess.run(learning_rate.assign(starter_learning_rate * ratio))
        #saver.save(sess, 'checkpoints/model.ckpt', epoch_n)
        # print( "Epoch ", epoch_n, ", Accuracy: ", sess.run(accuracy, feed_dict={inputs: mnist.test.images, outputs:mnist.test.labels, training: False}), "%" )
        #with open('train_log', 'a') as train_log:
        #    # write test accuracy to file "train_log"
        #    train_log_w = csv.writer(train_log)
        #    log_i = [epoch_n] + sess.run([accuracy], feed_dict={inputs: mnist.test.images, outputs: mnist.test.labels, training: False})
        #    train_log_w.writerow(log_i)

print( "Final Accuracy: ", sess.run(accuracy,
                                    feed_dict={
                                        inputs: mnist.semi_test.images,
                                        outputs: mnist.semi_test.labels,
                                        training: False}), "%" )

sess.close()

 13%|█▎        | 11417/90000 [02:55<26:29, 49.45it/s]

## show Result

## End